In [81]:
# Imports (External)
import numpy as np
import pandas as pd
import datetime
from datetime import datetime

from monthdelta import monthdelta
import xlrd
import xlsxwriter
from collections import OrderedDict
import pickle

import sys
sys.path.append('../')  

# Visualization/plotting imports
import matplotlib as mpl
import matplotlib.pyplot as plt

# Machine learning imports
import pywt
from pywt import wavedec, waverec
from scipy import signal
from statsmodels.robust import mad

import sklearn
import tensorflow as tf
import keras
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import optimizers
from keras import regularizers

# Internal Imports
from wsae_lstm.utils import pickle_load

In [82]:
dict_dataframes_index=pickle_load(path_filename="../data/interim/clean_data_index_interval.pickle")
#dict_dataframes_index.keys()

In [83]:
#dict_dataframes_index['csi300 index data'][1].drop(['date'],axis=1,inplace=True)

In [84]:
#dict_dataframes_index['csi300 index data'][1].reset_index(drop=True,inplace=True)

In [85]:
dict_dataframes_index['csi300 index data'][1].head()

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08
2008-07-03,2654.48,2807.68,2617.26,2760.61,456603.05,72.73,2.5762,-206.941406,-112.113057,190.424,3109.044731,2978.942155,2820.8364,-208.925,2753.2792,-82.064,-2.886857,-0.031251,-5.855737e+07
2008-07-04,2751.21,2783.85,2716.02,2741.85,379050.10,72.71,2.5632,-200.759162,-81.997539,67.825,3073.107115,2956.362140,2810.0548,-239.055,2738.4454,-249.421,-8.338289,-0.017237,-7.467503e+07
2008-07-07,2747.61,2890.99,2747.61,2882.76,527320.24,72.71,2.5679,-182.386907,21.707767,149.133,3046.256923,2949.352699,2819.3370,66.742,2756.6342,109.687,3.955427,0.010701,-3.531151e+07


In [86]:
dict_dataframes_index['csi300 index data'][1].head()

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08
2008-07-03,2654.48,2807.68,2617.26,2760.61,456603.05,72.73,2.5762,-206.941406,-112.113057,190.424,3109.044731,2978.942155,2820.8364,-208.925,2753.2792,-82.064,-2.886857,-0.031251,-5.855737e+07
2008-07-04,2751.21,2783.85,2716.02,2741.85,379050.10,72.71,2.5632,-200.759162,-81.997539,67.825,3073.107115,2956.362140,2810.0548,-239.055,2738.4454,-249.421,-8.338289,-0.017237,-7.467503e+07
2008-07-07,2747.61,2890.99,2747.61,2882.76,527320.24,72.71,2.5679,-182.386907,21.707767,149.133,3046.256923,2949.352699,2819.3370,66.742,2756.6342,109.687,3.955427,0.010701,-3.531151e+07


In [87]:
len(dict_dataframes_index['csi300 index data'][1])

612

In [88]:
len(dict_dataframes_index['csi300 index data'][1].columns)

19

In [89]:
len(dict_dataframes_index['csi300 index data'])

24

In [90]:
# train
dict_dataframes_index['csi300 index data'][1].index[0]

Timestamp('2008-07-01 00:00:00')

In [91]:
# validate
dict_dataframes_index['csi300 index data'][1].index[0] + monthdelta(24) 

Timestamp('2010-07-01 00:00:00')

In [92]:
# test
dict_dataframes_index['csi300 index data'][1].index[0] + monthdelta(27) 

Timestamp('2010-10-01 00:00:00')

In [93]:
dict_dataframes_index['csi300 index data'][1].index[0] + monthdelta(30)

Timestamp('2011-01-01 00:00:00')

In [107]:
def tvt_split(df):
    dict_dataframes = {}
    train = df.index[0]
    validate = df.index[0] + monthdelta(24) 
    test = df.index[0] + monthdelta(27) 
    test_end = df.index[0]  + monthdelta(30)
    
    df_train = pd.DataFrame(df[(df.index >= train) & (df.index <= validate)])
    df_validate = pd.DataFrame(df[(df.index >= validate) & (df.index <= test)])
    df_test = pd.DataFrame(df[(df.index >= test) & (df.index <= test_end)])
    dict_dataframes ={1:df_train,2:df_validate,3:df_test}
    return dict_dataframes

def dict_df_tvt_split(df):
    subdict_dataframes = {}
    for key in df:
        subdict_dataframes[key] =tvt_split(df[key])
    return subdict_dataframes

def dd_tvt_split(dict_dataframes):
    subdict_dataframes = {}
    for key in dict_dataframes:
        print(key)
        subdict_dataframes[key] = dict_df_tvt_split(dict_dataframes[key])
    return subdict_dataframes
tvt_split_df =dd_tvt_split(dict_dataframes_index)
type(tvt_split_df)

csi300 index data
nifty 50 index data
hangseng index data
nikkei 225 index data
s&p500 index data
djia index data


dict

In [108]:
len(tvt_split_df['csi300 index data'])

24

In [109]:
tvt_split_df['csi300 index data'][1][1].head(1)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2008-07-01,2799.2,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.2,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.77,-9.424605,-0.097927,-8.726278e+07


In [119]:
tvt_split_df['csi300 index data'][1][2].head(1)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2010-07-01,2557.92,2574.1,2524.57,2526.07,328576.46,84.55,2.2492,-68.405561,-199.061153,49.525,2742.562654,2713.585325,2691.0766,-232.424,2626.8448,-223.952,-8.143641,-0.063088,-9.298846e+07


In [123]:
tvt_split_df['csi300 index data'][1][3].head(1)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2010-10-08,2965.65,3055.84,2963.24,3044.23,999947.2,77.2,1.7325,14.071181,187.594016,120.262,2912.040962,2913.539251,2898.0211,194.405,2928.111,111.679,3.80825,0.043045,1.079573e+08


In [121]:
len(tvt_split_df['csi300 index data'][1][3])

61

In [128]:
dict_dataframes_index['csi300 index data'][1].head(700)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08
2008-07-03,2654.48,2807.68,2617.26,2760.61,456603.05,72.73,2.5762,-206.941406,-112.113057,190.424,3109.044731,2978.942155,2820.8364,-208.925,2753.2792,-82.064,-2.886857,-0.031251,-5.855737e+07
2008-07-04,2751.21,2783.85,2716.02,2741.85,379050.10,72.71,2.5632,-200.759162,-81.997539,67.825,3073.107115,2956.362140,2810.0548,-239.055,2738.4454,-249.421,-8.338289,-0.017237,-7.467503e+07
2008-07-07,2747.61,2890.99,2747.61,2882.76,527320.24,72.71,2.5679,-182.386907,21.707767,149.133,3046.256923,2949.352699,2819.3370,66.742,2756.6342,109.687,3.955427,0.010701,-3.531151e+07
2008-07-08,2900.57,2939.14,2841.07,2901.84,542480.78,72.96,2.5501,-164.392315,85.611186,98.070,3018.968885,2944.827775,2824.3296,110.022,2797.3330,52.173,1.830845,-0.022797,-2.815550e+07
2008-07-09,2927.00,3015.27,2927.00,3015.13,693569.68,72.59,2.5282,-139.383591,153.950657,113.430,2995.480423,2951.522844,2828.8885,316.779,2860.4388,225.185,8.071318,0.011349,5.866535e+07
2008-07-10,2970.98,3056.91,2950.20,2973.73,616495.54,72.49,2.4904,-121.503849,126.077089,106.705,2970.850308,2953.637621,2828.1706,274.131,2903.0620,121.813,4.271270,0.053033,6.811027e+07
2008-07-11,2962.86,2988.20,2913.87,2953.50,442871.01,71.91,2.3473,-107.724634,76.326934,74.325,2948.026538,2953.624323,2841.9183,192.886,2945.3912,-16.039,-0.540118,0.054742,7.257827e+07
